In [378]:
print("POR FAVOR, AGUARDE ALGUNS INSTANTES...")
print("")
print("--------------------------------------")
print("")
print("QUANDO A JANELA DE SELEÇÃO ABRIR, SELECIONE A PLANILHA DE REFERÊNCIA (arquivo enviado pelo Eduardo)")
print("")
print("--------------------------------------")
print("")
print("ANTES DE CONFIRMAR A EXECUÇÃO, CONFIRME SE OS PERÍODOS ESTÃO CONFORME O PADRÃO")
print("")
print("--------------------------------------")
print("")

POR FAVOR, AGUARDE ALGUNS INSTANTES...

--------------------------------------

QUANDO A JANELA DE SELEÇÃO ABRIR, SELECIONE A PLANILHA DE REFERÊNCIA (arquivo enviado pelo Eduardo)

--------------------------------------



In [379]:
import pandas as pd
import numpy as np
from tkinter import *                   # biblioteca para interação com o usuário
import tkinter.filedialog               # biblioteca para interação com o usuário
from tkinter import messagebox          # biblioteca para interação com o usuário
import warnings                         # biblioteca não mostrar mensagens de alerta do pandas
warnings.filterwarnings('ignore')       # Chamando a biblioteca não mostrar mensagens de alerta do pandas
from datetime import datetime, date     # Bibliteca para encontrar o dia 

In [905]:
# Dar a opção para que o usuário selecione o arquivo desejado
root= Tk()
nome_arquivo = tkinter.filedialog.askopenfilename(title = "Selecione o Arquivo de Referência (enviado pelo Eduardo)")
root.destroy()

In [666]:
print("POR FAVOR, AGUARDE ALGUNS INSTANTES...")
print("")
print("--------------------------------------")
print("")

POR FAVOR, AGUARDE ALGUNS INSTANTES...

--------------------------------------



In [906]:
# definição da variável com o arquivo selecionado pelo usuário
df =  pd.read_excel(nome_arquivo)

In [907]:
# Transformando a relação nominal dos responsáveis em lista para que seja utilizado como referência de filtro no comando abaixo
lista_nomes = list(df["RESPONSÁVEL"].unique())

In [908]:
# Solicitação que o usuário informe o seu nome para que o df seja referente somente a parte de sua responsabilidade
nome = input("QUAL O SEU NOME (conforme consta na primeira coluna da Planilha): ").upper()
# enquanto o usuário não escrever o nome corretamente, a mensagem se repetirá, até que o usuário escreva de forma correta
while nome not in lista_nomes:
            print("VERIFIQUE O NOME DIGITADO E TENTE NOVAMENTE")
            nome = input("QUAL O SEU NOME: ").upper()


QUAL O SEU NOME (conforme consta na primeira coluna da Planilha): romilda


In [909]:
# CPF e mês de referência têm a finalidade de acrescentar essa informação no aquivo final 
cpf = str(input("INFORME O NÚMERO DO SEU CPF (somente números): " ))
mes_ref = str(input("INFORME O ANO MÊS REFERÊNCIA (aaaamm): "))

INFORME O NÚMERO DO SEU CPF (somente números): 02274121176
INFORME O ANO MÊS REFERÊNCIA (aaaamm): 202208


In [910]:
# De acordo com o ano mes informado, será definido quantos dias o mês possui
if (mes_ref[4:] == "04") or (mes_ref[4:] == "06") or (mes_ref[4:] == "09") or (mes_ref[4:] == "11"):
    mes = 30
if (mes_ref[4:] == "01") or (mes_ref[4:] == "03") or (mes_ref[4:] == "05") or (mes_ref[4:] == "07") or (mes_ref[4:] == "08") or (mes_ref[4:] == "10")or (mes_ref[4:] == "12"):
    mes = 31
if (mes_ref[4:] == "02"):
    mes = 28

In [911]:
# filtrando pelo nome para que o df seja referente somente a parte de sua responsabilidade
df = df.loc[df["RESPONSÁVEL"] == nome, ["NOME", "MATRÍCULA", "PERÍODO DO TELETRABALHO (De acordo com o Relatório de Frequência SEI enviado pela Unidade)", "SITUAÇÃO SIAPE"]].reset_index()
# se houver alguma coisa escrita na coluna SITAÇÃO SIAPE, o programa não fará a leitura da linha
# isso serve para que, caso o usuário já tenha feito o registro anteriormente, não seja feito novamente
df = df.loc[df["SITUAÇÃO SIAPE"].isnull(), :]

In [912]:
# renomeando colunas
df.rename(columns={"MATRÍCULA":"MATRICULA", 
                  "PERÍODO DO TELETRABALHO (De acordo com o Relatório de Frequência SEI enviado pela Unidade)": "PERIODO"}, inplace=True)

In [913]:
# armazenando o nome e matrícula do servidor para fazer o "merge" mais a frente
armazem_nome = df[["NOME", "MATRICULA"]]

In [914]:
# excluindo colunas desnecessárias 
df = df.drop(["index","SITUAÇÃO SIAPE"], axis = 1) 

In [915]:
# Considerando a necessidade de manter um padrão na quantidade total de dias no mês, cria-se, temporariamente, um novo servidor
# Neste, por sua vez, constará registro de afastamento todos os dias do mês. A quantidade varia, conforme o mês (29, 30, 31) 
temp = pd.DataFrame([{"NOME": "xxxxxxx","MATRICULA": 9999999,"PERIODO": str(list(np.arange(1,mes+1)))}])
# Considerando a o resultdo do PERÍODO é uma lista, retira-se os colhetes
temp["PERIODO"] = temp["PERIODO"].str.replace("[", "")
temp["PERIODO"] = temp["PERIODO"].str.replace("]", "")

In [916]:
# A coluna PERÍODO pode ter várias formas de apresentação. A fim de padronizar o inervalo dos dias, toda separação será por ","
df["PERIODO"] = df["PERIODO"].str.replace("DE", "")
df["PERIODO"] = df["PERIODO"].str.replace("De", "")
df["PERIODO"] = df["PERIODO"].str.replace("de", "")
df["PERIODO"] = df["PERIODO"].str.replace("dE", "")
df["PERIODO"] = df["PERIODO"].str.replace("EM", "")
df["PERIODO"] = df["PERIODO"].str.replace("Em", "")
df["PERIODO"] = df["PERIODO"].str.replace("em", "")
df["PERIODO"] = df["PERIODO"].str.replace(";", ",")
df["PERIODO"] = df["PERIODO"].str.replace("e", ",")
df["PERIODO"] = df["PERIODO"].str.replace("E", ",")
df["PERIODO"] = df["PERIODO"].str.replace(".", ",")
df["PERIODO"] = df["PERIODO"].str.replace("'", "")


In [917]:
# Para garantir que o split (separação) funcione corretamente, faz-se a união entre o arquivo temporário (servidor fictício)
# com o df original (que nem sempre terá todos os dias do mês para registrar)
df = pd.concat([temp,df])

In [918]:
# O Split será feito em quantitade de acordo com o número de dias do mês. A Separação considera a "," como referência
if mes == 31:
    df[[1,2,3,4,5,6,7,8,9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]] = df["PERIODO"].str.split(",", expand=True)

if mes == 30:
    df[[1,2,3,4,5,6,7,8,9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]] = df["PERIODO"].str.split(",", expand=True)

if mes == 29:
    df[[1,2,3,4,5,6,7,8,9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]] = df["PERIODO"].str.split(",", expand=True)

if mes == 28:
    df[[1,2,3,4,5,6,7,8,9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]] = df["PERIODO"].str.split(",", expand=True)

In [919]:
# Exclusão de colunas desnecessárias
df = df.drop(["NOME", "PERIODO"], axis = 1) 

In [920]:
# Transformando as colunas, que já possuem em cada uma um único dia, em uma única coluna 
df = pd.melt(df, id_vars = ["MATRICULA"], value_vars = df.columns, var_name = "PERIODO", value_name = "Dia Inicio")

In [921]:
# Retirando espaços indesejados (início ou fim de cada valor)
df["Dia Inicio"] = df["Dia Inicio"].str.strip()

In [922]:
# Retirando espaços entre os valores para padronização
df["Dia Inicio"] = df["Dia Inicio"].str.replace(" ", "")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("  ", "")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("   ", "")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("    ", "")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("     ", "")

In [923]:
# Considerando que nem sempre haverá "/" na informação, neste momento, estamos criando data com barras para que não ocorra erro
# O erro é possível de aconteler no momento do split (mais a frente), caso não encontre nenhuma barra (poderia utilizar o Try?)
temp2 = pd.DataFrame([{"MATRICULA": 9999999,"Dia Inicio": "01/01/11"}])

In [924]:
# Para garantir que o split (separação) funcione corretamente, faz-se a união entre o arquivo temporário e o df original
df = pd.concat([temp2,df])

In [925]:
# realização do split, considerando a "/" como delimitador
df[["Dia Inicio", "excluir1", "excluir2"]] = df["Dia Inicio"].str.split("/", expand=True)

In [926]:
# Exclusão de colunas desnecessárias, que são as informações após as barras (que, via de regra, são o mês e/ou o ano)
df = df.drop(["excluir1", "excluir2"], axis = 1) 

In [927]:
# Para o resultado final é preciso ter a informaçao do "0" antes do dígito menor que 10, realiza-se a substituição
df.loc[df["Dia Inicio"] == "1", "Dia Inicio"] = "01"
df.loc[df["Dia Inicio"] == "2", "Dia Inicio"] = "02"
df.loc[df["Dia Inicio"] == "3", "Dia Inicio"] = "03"
df.loc[df["Dia Inicio"] == "4", "Dia Inicio"] = "04"
df.loc[df["Dia Inicio"] == "5", "Dia Inicio"] = "05"
df.loc[df["Dia Inicio"] == "6", "Dia Inicio"] = "06"
df.loc[df["Dia Inicio"] == "7", "Dia Inicio"] = "07"
df.loc[df["Dia Inicio"] == "8", "Dia Inicio"] = "08"
df.loc[df["Dia Inicio"] == "9", "Dia Inicio"] = "09"

In [928]:
# padronização para que todas as letras fiquem minúsculas
df["Dia Inicio"] = df["Dia Inicio"].str.replace("A", "a")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("À", "a")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("Á", "a")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("á", "a")
df["Dia Inicio"] = df["Dia Inicio"].str.replace("à", "a")

In [929]:
# criação de df somente com os dias que possuem o "a" para separação
df2 = df.drop(df.loc[(df["Dia Inicio"] == "01")|(df["Dia Inicio"] == "02")|(df["Dia Inicio"] == "03")|(df["Dia Inicio"] == "04")|
            (df["Dia Inicio"] == "05")|(df["Dia Inicio"] == "06")|(df["Dia Inicio"] == "07")|(df["Dia Inicio"] == "08")|
            (df["Dia Inicio"] == "09")|(df["Dia Inicio"] == "10")|(df["Dia Inicio"] == "11")|(df["Dia Inicio"] == "12")|
            (df["Dia Inicio"] == "13")|(df["Dia Inicio"] == "14")|(df["Dia Inicio"] == "15")|(df["Dia Inicio"] == "16")|
            (df["Dia Inicio"] == "17")|(df["Dia Inicio"] == "18")|(df["Dia Inicio"] == "19")|(df["Dia Inicio"] == "20")|
            (df["Dia Inicio"] == "21")|(df["Dia Inicio"] == "22")|(df["Dia Inicio"] == "23")|(df["Dia Inicio"] == "24")|
            (df["Dia Inicio"] == "25")|(df["Dia Inicio"] == "26")|(df["Dia Inicio"] == "27")|(df["Dia Inicio"] == "28")|
            (df["Dia Inicio"] == "29")|(df["Dia Inicio"] == "30")|(df["Dia Inicio"] == "31")
            ].index)

In [930]:
# expandindo e seperando em cada coluna os dias que estavam com "a"
df2[["Dia Inicio","Dia Fim"]] =df2["Dia Inicio"].str.split("a", expand=True)

In [931]:
# Para o resultado final é preciso ter a informaçao do "0" antes do dígito menor que 10, realiza-se a substituição
df2.loc[df2["Dia Inicio"] == "1", "Dia Inicio"] = "01"
df2.loc[df2["Dia Inicio"] == "2", "Dia Inicio"] = "02"
df2.loc[df2["Dia Inicio"] == "3", "Dia Inicio"] = "03"
df2.loc[df2["Dia Inicio"] == "4", "Dia Inicio"] = "04"
df2.loc[df2["Dia Inicio"] == "5", "Dia Inicio"] = "05"
df2.loc[df2["Dia Inicio"] == "6", "Dia Inicio"] = "06"
df2.loc[df2["Dia Inicio"] == "7", "Dia Inicio"] = "07"
df2.loc[df2["Dia Inicio"] == "8", "Dia Inicio"] = "08"
df2.loc[df2["Dia Inicio"] == "9", "Dia Inicio"] = "09"
df2.loc[df2["Dia Fim"] == "1", "Dia Fim"] = "01"
df2.loc[df2["Dia Fim"] == "2", "Dia Fim"] = "02"
df2.loc[df2["Dia Fim"] == "3", "Dia Fim"] = "03"
df2.loc[df2["Dia Fim"] == "4", "Dia Fim"] = "04"
df2.loc[df2["Dia Fim"] == "5", "Dia Fim"] = "05"
df2.loc[df2["Dia Fim"] == "6", "Dia Fim"] = "06"
df2.loc[df2["Dia Fim"] == "7", "Dia Fim"] = "07"
df2.loc[df2["Dia Fim"] == "8", "Dia Fim"] = "08"
df2.loc[df2["Dia Fim"] == "9", "Dia Fim"] = "09"

In [932]:
# criação de df somente com os números que estão individuais e serão repetidos na data fim
df = df.loc[
            (df["Dia Inicio"] == "01")|(df["Dia Inicio"] == "02")|(df["Dia Inicio"] == "03")|(df["Dia Inicio"] == "04")|
            (df["Dia Inicio"] == "05")|(df["Dia Inicio"] == "06")|(df["Dia Inicio"] == "07")|(df["Dia Inicio"] == "08")|
            (df["Dia Inicio"] == "09")|(df["Dia Inicio"] == "10")|(df["Dia Inicio"] == "11")|(df["Dia Inicio"] == "12")|
            (df["Dia Inicio"] == "13")|(df["Dia Inicio"] == "14")|(df["Dia Inicio"] == "15")|(df["Dia Inicio"] == "16")|
            (df["Dia Inicio"] == "17")|(df["Dia Inicio"] == "18")|(df["Dia Inicio"] == "19")|(df["Dia Inicio"] == "20")|
            (df["Dia Inicio"] == "21")|(df["Dia Inicio"] == "22")|(df["Dia Inicio"] == "23")|(df["Dia Inicio"] == "24")|
            (df["Dia Inicio"] == "25")|(df["Dia Inicio"] == "26")|(df["Dia Inicio"] == "27")|(df["Dia Inicio"] == "28")|
            (df["Dia Inicio"] == "29")|(df["Dia Inicio"] == "30")|(df["Dia Inicio"] == "31")]

In [933]:
# Como cada dia será um registro e é preciso informar o dia início e o dia fim, faz-se uma cópia do dia início (q tb é o fim)
df["Dia Fim"] = df["Dia Inicio"]

In [934]:
# Após o tratamento dos que tinham ou não o "a",junta-se os dois DFs
df = pd.concat([df, df2])

In [935]:
# Ordenação dos valores
df = df.sort_values(by=["MATRICULA", "PERIODO", "Dia Inicio"])

In [936]:
# Retirando espaços indesejados, que podem ter aparecido após o último split
df["Dia Inicio"] = df["Dia Inicio"].str.strip()
df["Dia Fim"] = df["Dia Fim"].str.strip()

In [937]:
# Para o resultado final, espera-se somente número (de 01 a 31 no máximo). Qualquer valor fora disso é excluído
df = df.loc[
            (df["Dia Inicio"] == "01")|(df["Dia Inicio"] == "02")|(df["Dia Inicio"] == "03")|(df["Dia Inicio"] == "04")|
            (df["Dia Inicio"] == "05")|(df["Dia Inicio"] == "06")|(df["Dia Inicio"] == "07")|(df["Dia Inicio"] == "08")|
            (df["Dia Inicio"] == "09")|(df["Dia Inicio"] == "10")|(df["Dia Inicio"] == "11")|(df["Dia Inicio"] == "12")|
            (df["Dia Inicio"] == "13")|(df["Dia Inicio"] == "14")|(df["Dia Inicio"] == "15")|(df["Dia Inicio"] == "16")|
            (df["Dia Inicio"] == "17")|(df["Dia Inicio"] == "18")|(df["Dia Inicio"] == "19")|(df["Dia Inicio"] == "20")|
            (df["Dia Inicio"] == "21")|(df["Dia Inicio"] == "22")|(df["Dia Inicio"] == "23")|(df["Dia Inicio"] == "24")|
            (df["Dia Inicio"] == "25")|(df["Dia Inicio"] == "26")|(df["Dia Inicio"] == "27")|(df["Dia Inicio"] == "28")|
            (df["Dia Inicio"] == "29")|(df["Dia Inicio"] == "30")|(df["Dia Inicio"] == "31")|
            (df["Dia Inicio"] == 1)|(df["Dia Inicio"] == 2)|(df["Dia Inicio"] == 3)|(df["Dia Inicio"] == 4)|
            (df["Dia Inicio"] == 5)|(df["Dia Inicio"] == 6)|(df["Dia Inicio"] == 7)|(df["Dia Inicio"] == 8)|
            (df["Dia Inicio"] == 9)|(df["Dia Inicio"] == 10)|(df["Dia Inicio"] == 11)|(df["Dia Inicio"] == 12)|
            (df["Dia Inicio"] == 13)|(df["Dia Inicio"] == 14)|(df["Dia Inicio"] == 15)|(df["Dia Inicio"] == 16)|
            (df["Dia Inicio"] == 17)|(df["Dia Inicio"] == 18)|(df["Dia Inicio"] == 19)|(df["Dia Inicio"] == 20)|
            (df["Dia Inicio"] == 21)|(df["Dia Inicio"] == 22)|(df["Dia Inicio"] == 23)|(df["Dia Inicio"] == 24)|
            (df["Dia Inicio"] == 25)|(df["Dia Inicio"] == 26)|(df["Dia Inicio"] == 27)|(df["Dia Inicio"] == 28)|
            (df["Dia Inicio"] == 29)|(df["Dia Inicio"] == 30)|(df["Dia Inicio"] == 31)
           ]

In [938]:
# Retirando espaços indesejados (novamente)
df["Dia Inicio"] = df["Dia Inicio"].str.strip()
df["Dia Fim"] = df["Dia Fim"].str.strip()

In [939]:
# para visualizar o nome do servidor e confirmar a informação, faz-se um "merge" com os nomes guardados anteriormente
df = pd.merge(df, armazem_nome, on="MATRICULA", how="left")

In [940]:
# Nova ordenação 
df = df.sort_values(by=["NOME", "Dia Inicio"])

In [941]:
# inclusão de colunas com informações necessárias para a elaboração do arquivo final
df["Padrao"] = "1"
df["CPF"] = cpf
df["ORGAO"] = "37000"
df["Ano_mes_inicio"] = mes_ref
df["Ano_mes_fim"] = mes_ref
df["Cod_Afast"] = "0390"


In [942]:
# Ordenação das colunas
df = df[["NOME", "Padrao", "CPF", "ORGAO", "MATRICULA", "Ano_mes_inicio", "Dia Inicio",
        "Ano_mes_fim", "Dia Fim", "Cod_Afast"]]

In [943]:
# Transformando o tipo da coluna matrícula para possibilitar a inclusão dos "0"s naquelas que possuem menos que 7 dígitos
df["MATRICULA"] = df["MATRICULA"].astype("str")

In [944]:
# inclusão de "0"s a esquerda nas matrículas que possuem menos do que 7 dígitos
 
for i in df.index:
    if len(df["MATRICULA"][i]) == 6:
        df["MATRICULA"][i] = df["MATRICULA"][i].replace(df["MATRICULA"][i], "0" + df["MATRICULA"][i])
    elif len(df["MATRICULA"][i]) == 5:
        df["MATRICULA"][i] = df["MATRICULA"][i].replace(df["MATRICULA"][i], "00" + df["MATRICULA"][i])
    elif len(df["MATRICULA"][i]) == 4:
        df["MATRICULA"][i] = df["MATRICULA"][i].replace(df["MATRICULA"][i], "000" + df["MATRICULA"][i])    
    else:
        pass

In [945]:
if mes == 31:
    df.loc[(df["Dia Inicio"] == "01") & (df["Dia Fim"] == "31"), "Cod_Afast"] = "0389"
if mes == 30:
    df.loc[(df["Dia Inicio"] == "01") & (df["Dia Fim"] == "30"), "Cod_Afast"] = "0389"
if mes == 28:
    df.loc[(df["Dia Inicio"] == "01") & (df["Dia Fim"] == "28"), "Cod_Afast"] = "0389"

In [946]:
# Criação da coluna TXT, que é o resultdo final e objetivo do programa
df["TXT"] = df["Padrao"] + df["CPF"] +df["ORGAO"] +df["MATRICULA"] +df["Ano_mes_inicio"] + df["Dia Inicio"] +df["Ano_mes_fim"] + df["Dia Fim"] + df["Cod_Afast"]

In [947]:
# Definição da data do dia da execução do programa
today = date.today().strftime("%d/%m/%Y")
today = today.replace("/", "-")

In [948]:
# Exclusão de todos os registros do servidor fictício criado somente para estabeler os tamanhos e informações padrões
df = df.drop(df.loc[df["MATRICULA"] == "9999999"].index)

In [949]:
# Exportando o arquivo final. O nome do arquivo será salvo considerando as variáveis nome do Responsável, o ano mes, e a data
df.to_excel("Carga Batch {} {} em {}.xlsx".format(nome,mes_ref,today), index=False)

In [802]:
# definindo a quantidade de linhas que foram geradas
qtd_linhas = df.shape[0]

In [456]:
# printando informações para o usuário

print("LEITURA REALIZADA COM SUCESSO, ARQUIVO EM EXCEL SALVO COM O SEGUINTE NOME: Carga Batch {} {} em {}.xlsx".format(nome,mes_ref,today))
print("")
print("-------------------------------------------")
print("")
print("PRIMEIRA LINHA DO ARQUIVO TXT: 037000{}000".format(mes_ref))
print("")
print("-------------------------------------------")
print("")
if qtd_linhas < 100:
    print("ULTIMA LINHA DO ARQUIVO TXT: 90000{}".format(qtd_linhas))
if qtd_linhas > 100 and qtd_linhas < 1000:
    print("ULTIMA LINHA DO ARQUIVO TXT: 9000{}".format(qtd_linhas))
if qtd_linhas > 1000 and qtd_linhas < 10000:
    print("ULTIMA LINHA DO ARQUIVO TXT: 900{}".format(qtd_linhas))

LEITURA REALIZADA COM SUCESSO, ARQUIVO EM EXCEL SALVO COM O SEGUINTE NOME: Carga Batch ADRIANO 202208 em 14-10-2022.xlsx

-------------------------------------------

PRIMEIRA LINHA DO ARQUIVO TXT: 037000202208000

-------------------------------------------

ULTIMA LINHA DO ARQUIVO TXT: 9000017


In [ ]:
# exibindo informações para o usuário 
root= Tk()
messagebox.showinfo("LEITURA REALIZADA COM SUCESSO", "ARQUIVO EM EXCEL SALVO COM O SEGUINTE NOME \n Carga Batch {} {} em {}.xlsx".format(nome,mes_ref,today))
messagebox.showinfo("PRIMEIRA LINHA DO ARQUIVO TXT:", " \n 037000{}000".format(mes_ref))
if qtd_linhas < 100:
    messagebox.showinfo("ULTIMA LINHA DO ARQUIVO TXT", "\n 90000{}".format(qtd_linhas))
if qtd_linhas > 100 and qtd_linhas < 1000:
    messagebox.showinfo("ULTIMA LINHA DO ARQUIVO TXT", "\n 9000{}".format(qtd_linhas))
if qtd_linhas > 1000 and qtd_linhas < 10000:
    messagebox.showinfo("ULTIMA LINHA DO ARQUIVO TXT", "\n 900{}".format(qtd_linhas))    
root.destroy()